In [1]:
using Pkg
Pkg.add(["CSV", "DataFrames", "Gurobi", "Distances"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [36]:
using CSV
using DataFrames
using Gurobi
using Random
using Distances
using JuMP
using Gurobi


# Load data
pairs = CSV.read("./data/kidney_data_with_states.csv", DataFrame)
centers = CSV.read("./data/center_small.csv", DataFrame)

# # Take random samples
pair_size = 2000
# center_size = 6
Random.seed!(42)
pairs = pairs[shuffle(1:nrow(pairs))[1:min(pair_size, nrow(pairs))], :]
# centers = centers[shuffle(1:nrow(centers))[1:min(center_size, nrow(centers))], :]

n = nrow(pairs)
m = nrow(centers)

println("Number of pairs: ", n)
println("Number of centers: ", m)
    

# Define blood type compatibility scores
blood_type_scores = Dict(
    ("O", "O") => 10, ("O", "A") => 5, ("O", "B") => 5, ("O", "AB") => 3,
    ("A", "A") => 10, ("A", "AB") => 5,
    ("B", "B") => 10, ("B", "AB") => 5,
    ("AB", "AB") => 10
)

# Calculate matching scores
matching_scores = Dict()
for i in 1:n, j in 1:n
    donor = pairs[i, :Donor]
    patient = pairs[j, :Patient]
    matching_scores[(i, j)] = get(blood_type_scores, (donor, patient), 0)
end

# Calculate d_max and α as before
d_max = 0.0
for i in 1:n, j in 1:n, k in 1:m
    total_distance = distances[(i,k)] + distances[(j,k)]
    global d_max = max(d_max, total_distance)
end

α = 1.0 / (2 * d_max * n)

# Modified objective function incorporating matching scores
@objective(model, Max, 
    sum(matching_scores[(i,j)] * x[i,j,k] for i in 1:n for j in 1:n for k in 1:m) -
    α * sum((distances[(i,k)] + distances[(j,k)]) * x[i,j,k] for i in 1:n for j in 1:n for k in 1:m)
)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-03
Number of pairs: 2000
Number of centers: 6
Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-03
Maximum distance in sampled data (d_max): 18395.07 km
Calculated α: 0


In [37]:
# Create model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[1:n, 1:n, 1:m], Bin)  # x[i,j,k] = 1 if donor i donates to patient j through center k
@variable(model, y[1:m], Bin)  # y[k] = 1 if center k is used


# Objective function
@objective(model, Max, 
    sum(x[i,j,k] for i in 1:n for j in 1:n for k in 1:m) -
    α * sum((distances[(i,k)] + distances[(j,k)]) * x[i,j,k] for i in 1:n for j in 1:n for k in 1:m)
)

# Constraints
# 1. Donor limit
for i in 1:n
    @constraint(model, sum(x[i,j,k] for j in 1:n for k in 1:m) <= 1)
end

# 2. Patient limit
for j in 1:n
    @constraint(model, sum(x[i,j,k] for i in 1:n for k in 1:m) <= 1)
end

# 3. Compatibility constraint
for i in 1:n, j in 1:n, k in 1:m
    @constraint(model, x[i,j,k] <= comp[(i,j)])
end

# 4. Transplant center capacity
for k in 1:m
    capacity = centers[k, Symbol("Living Donor Transplants in a year")]
    @constraint(model, sum(x[i,j,k] for i in 1:n for j in 1:n) <= capacity * y[k])
end

# 5. Center usage constraint
for k in 1:m
    @constraint(model, y[k] >= sum(x[i,j,k] for i in 1:n for j in 1:n) / (n * n))
end

# Optimize
optimize!(model)

# Print results
if termination_status(model) == MOI.OPTIMAL
    println("\nOptimal Solution Found:")
    for i in 1:n, j in 1:n, k in 1:m
        if value(x[i,j,k]) > 0.5
            println("Donor $i donates to Patient $j through Center $k")
        end
    end
    
    used_centers = [k for k in 1:m if value(y[k]) > 0.5]
    println("\nSelected Transplant Centers: ", [centers[k, Symbol("mix-text_weightBold")] for k in used_centers])
    
    total_exchanges = sum(value(x[i,j,k]) for i in 1:n for j in 1:n for k in 1:m)
    total_travel_distance = sum((distances[(i,k)] + distances[(j,k)]) * value(x[i,j,k]) 
                               for i in 1:n for j in 1:n for k in 1:m)
    
    println("\nTotal Exchanges: ", total_exchanges)
    println("Total Travel Distance: ", round(total_travel_distance, digits=2), " km")
    println("Objective Value: ", round(objective_value(model), digits=2))
else
    println("No optimal solution found.")
end


In [17]:
# Create DataFrames to store results if optimization was successful
if termination_status(model) == MOI.OPTIMAL
    # Save matching results
    matches_df = DataFrame(
        Donor_ID = Int[],
        Recipient_ID = Int[],
        Center_ID = Int[],
        Donor_Lat = Float64[],
        Donor_Lng = Float64[],
        Recipient_Lat = Float64[],
        Recipient_Lng = Float64[],
        Center_Lat = Float64[],
        Center_Lng = Float64[],
        Distance = Float64[]
    )

    # Collect all successful matches
    for i in 1:n, j in 1:n, k in 1:m
        if value(x[i,j,k]) > 0.5
            total_distance = distances[(i,k)] + distances[(j,k)]
            push!(matches_df, [
                i, j, k,
                pairs[i, :lat], pairs[i, :lng],
                pairs[j, :lat], pairs[j, :lng],
                centers[k, :lat], centers[k, :lng],
                total_distance
            ])
        end
    end

    # Save centers usage
    centers_df = DataFrame(
        Center_ID = Int[],
        Name = String[],
        Lat = Float64[],
        Lng = Float64[],
        Is_Used = Bool[]
    )

    for k in 1:m
        push!(centers_df, [
            k,
            centers[k, Symbol("mix-text_weightBold")],
            centers[k, :lat],
            centers[k, :lng],
            value(y[k]) > 0.5
        ])
    end

    # Save to CSV files
    CSV.write("kidney_matches.csv", matches_df)
    CSV.write("centers_usage.csv", centers_df)
    
    println("Results have been saved to 'kidney_matches.csv' and 'centers_usage.csv'")
end

Results have been saved to 'kidney_matches.csv' and 'centers_usage.csv'
